In [1]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split

In [9]:
# from our acquire.py:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
sql_query = query = '''
SELECT
    prop.*,
    predictions_2017.logerror,
    predictions_2017.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
FROM properties_2017 prop
JOIN (
    SELECT parcelid, MAX(transactiondate) AS max_transactiondate
    FROM predictions_2017
    GROUP BY parcelid
) pred USING(parcelid)
JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                      AND pred.max_transactiondate = predictions_2017.transactiondate
LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
LEFT JOIN storytype story USING (storytypeid)
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE prop.latitude IS NOT NULL
  AND prop.longitude IS NOT NULL
  AND transactiondate <= '2017-12-31'
'''
    
def get_zillow_data():
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df = df.drop(columns='id')
    return df

In [10]:
df = get_zillow_data()

In [11]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,NaN,...,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,NaN,...,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,NaN,...,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,NaN,...,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,10887214,1.00,NaN,NaN,3.00,3.00,NaN,8.00,3.00,NaN,...,"60,371,236,012,000.00",0.01,2017-01-01,Central,None,None,Central,Condominium,None,None


### Summary Statistics

In [12]:
df.info()
#Write some stuff tomorrow

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77380 non-null  int64  
 1   airconditioningtypeid         24953 non-null  float64
 2   architecturalstyletypeid      206 non-null    float64
 3   basementsqft                  50 non-null     float64
 4   bathroomcnt                   77380 non-null  float64
 5   bedroomcnt                    77380 non-null  float64
 6   buildingclasstypeid           15 non-null     float64
 7   buildingqualitytypeid         49671 non-null  float64
 8   calculatedbathnbr             76771 non-null  float64
 9   decktypeid                    614 non-null    float64
 10  finishedfloor1squarefeet      6023 non-null   float64
 11  calculatedfinishedsquarefeet  77184 non-null  float64
 12  finishedsquarefeet12          73748 non-null  float64
 13  f

In [13]:
df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,"77,380.00","24,953.00",206.00,50.00,"77,380.00","77,380.00",15.00,"49,671.00","76,771.00",614.00,...,"17,560.00",172.00,"77,268.00","77,379.00","77,380.00","77,378.00","77,375.00","2,886.00","77,136.00","77,380.00"
mean,"13,007,149.52",1.81,7.39,679.72,2.30,3.05,3.93,6.53,2.32,66.00,...,1.43,1.00,"189,315.98","490,137.46","2,016.00","301,096.94","5,995.57",14.09,"60,496,735,236,341.99",0.02
std,"3,481,368.13",2.97,2.73,689.70,1.00,1.14,0.26,1.72,0.98,0.00,...,0.54,0.00,"230,088.56","653,447.67",0.00,"492,599.03","7,622.89",2.19,"1,535,251,869,244.56",0.17
min,"10,711,855.00",1.00,2.00,38.00,0.00,0.00,3.00,1.00,1.00,66.00,...,1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",-4.66
25%,"11,538,296.75",1.00,7.00,273.00,2.00,2.00,4.00,6.00,2.00,66.00,...,1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.59",14.00,"60,373,109,005,001.75",-0.02
50%,"12,531,548.50",1.00,7.00,515.00,2.00,3.00,4.00,6.00,2.00,66.00,...,1.00,1.00,"136,499.50","358,976.00","2,016.00","203,383.50","4,450.72",15.00,"60,376,032,003,008.00",0.01
75%,"14,211,843.75",1.00,7.00,796.50,3.00,4.00,4.00,8.00,3.00,66.00,...,2.00,1.00,"218,787.50","569,003.00","2,016.00","366,802.25","6,927.80",15.00,"60,590,423,251,008.00",0.04
max,"167,689,317.00",13.00,21.00,"3,560.00",18.00,16.00,4.00,12.00,18.00,66.00,...,6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00",5.26


In [14]:
df.shape

(77380, 67)

In [15]:
df.columns.value_counts().isna()

parcelid                      False
structuretaxvaluedollarcnt    False
regionidcity                  False
regionidcounty                False
regionidneighborhood          False
                              ...  
poolsizesum                   False
pooltypeid10                  False
pooltypeid2                   False
pooltypeid7                   False
typeconstructiondesc          False
Length: 67, dtype: bool

In [23]:
def missing_rows_count_percentage(df):

    # This function returns number of rows missing and their percentage

    data = pd.DataFrame({
        'row_count': df.isna().sum(),
        'row_percent': df.isna().mean(),
    })

    return data

In [27]:
missing_rows_count_percentage(df).head()

,row_count,row_percent
parcelid,0,0.00
airconditioningtypeid,52427,0.68
architecturalstyletypeid,77174,1.00
basementsqft,77330,1.00
bathroomcnt,0,0.00
